### Nama : Fauzan Pradana
### NIM : 2041720224

# Naive Bayes dengan Data Kontinu

Pada percobaan ketiga ini, kita akan menggunakan data riil untuk melakukan klasifikasi dengan Naive Bayes. Data yang digunakan adalah **titanic.csv**. 

## Tahap Persiapan
Pada tahap ini kita akan melakukan beberapa hal,

1. Load data ke dalam data frame
2. Memisahkan fitur dan label
3. Split data untuk training dan testing

In [83]:
import numpy as np
import pandas as pd

# Load data CSV
data = pd.read_csv('dataset/titanic.csv')

# Drop the unnecessary columns
df = data.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'], axis='columns')

# Cek data
display(df.head())

# Overview of the data
df.describe(include='all')

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


,Pclass,Sex,Age,Fare,Survived
count,891.000000,891,714.000000,891.000000,891.000000
unique,NaN,2,NaN,NaN,NaN
top,NaN,male,NaN,NaN,NaN
freq,NaN,577,NaN,NaN,NaN
mean,2.308642,NaN,29.699118,32.204208,0.383838
std,0.836071,NaN,14.526497,49.693429,0.486592
min,1.000000,NaN,0.420000,0.000000,0.000000
25%,2.000000,NaN,20.125000,7.910400,0.000000
50%,3.000000,NaN,28.000000,14.454200,0.000000
75%,3.000000,NaN,38.000000,31.000000,1.000000


In [84]:
# Encoding
# Fungsi encoding yang akan digunakan adalah LabelEncoder
# Hal ini karena kita hanya mengganti nilai variabel dari nama berupa string menjadi angka. Sama halnya dengan label

from sklearn.preprocessing import LabelEncoder

# Inisiasi label encoder
encode = LabelEncoder()

# Terpakan label encoder
df['Sex'] = encode.fit_transform(df['Sex'])

# Cek hasil
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,1,22.0,7.2500,0
1,1,0,38.0,71.2833,1
2,3,0,26.0,7.9250,1
3,1,0,35.0,53.1000,1
4,3,1,35.0,8.0500,0


In [85]:
# missing value
mv = df.isnull().sum()
mv

Pclass        0
Sex           0
Age         177
Fare          0
Survived      0
dtype: int64

In [86]:
# Remove all the data that has missing values
df_no_mv = df.dropna(axis=0)
df_no_mv.describe(include='all')

,Pclass,Sex,Age,Fare,Survived
count,714.000000,714.000000,714.000000,714.000000,714.000000
mean,2.236695,0.634454,29.699118,34.694514,0.406162
std,0.838250,0.481921,14.526497,52.918930,0.491460
min,1.000000,0.000000,0.420000,0.000000,0.000000
25%,1.000000,0.000000,20.125000,8.050000,0.000000
50%,2.000000,1.000000,28.000000,15.741700,0.000000
75%,3.000000,1.000000,38.000000,33.375000,1.000000
max,3.000000,1.000000,80.000000,512.329200,1.000000


In [87]:
# Memisahkan fitur dengan label
X = df_no_mv.iloc[:,:-1]
y = df_no_mv.iloc[:,-1]

In [88]:
# Split data training dan testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=30)

## Training dan Evaluasi Model

In [89]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Inisiasi obyek MultinomialNB
gnb = GaussianNB()

# Fit model
# Label y harus dalam bentu 1D atau (n_samples,)
gnb.fit(X_train, y_train)

# Prediksi dengan data training
y_train_pred = gnb.predict(X_train)

# Evaluasi akurasi training
acc_train = accuracy_score(y_train, y_train_pred)

# Prediksi test data
y_test_pred = gnb.predict(X_test)

# Evaluasi model dengan metric akurasi
acc_test = accuracy_score(y_test, y_test_pred)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.7895791583166333
Hasil akurasi data test: 0.7534883720930232


## Apakah kita telah melakukan hal yang benar?

## Pembuktian

Kita akan melakukan percobaan lanjutan dengan menggunakan standarisasi untuk menjawab pertanyaan, apakah kita perlu melakukan hal tersebut pada model Naive Bayes khususnya tipe Gaussian.

In [90]:
from sklearn.preprocessing import StandardScaler

# Inisasi obyek StandardScaler
scaler = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

***Pertanyaan***

Mengapa pada X_test kita tidak perlu melakukan proses fitting?

***Jawaban***

Proses fitting (fit) akan menyimpan perhitungan berdasarkan data yang dilatih. Pada kasus standarisasi ini, nilai yang disimpan adalah *mean* dan *standar deviasi* dari data **X_train**. Jika kita melakukan proses fitting kembali pada **X_test** maka nilai mean dan standar deviasi akan berdasarkan data X_test. Kita tidak ingin itu terjadi, karena model diharapkan mampu melakukan klasifikasi dengan baik pada data yang tidak diketahui (data test). Oleh karena itu, pada X_test hanya dilakukan proses **transform** agar pada saat pembuatan model, model akan menggunakan nilai mean dan standar deviasi yang sama dengan data training.

In [91]:
# Buat obyek GaussianNB lain
gnb_std = GaussianNB()

# Fit dengan data yang telah di standarisasi
gnb_std.fit(X_train_std, y_train)

# Prediksi dengan data training
y_train_std_pred = gnb_std.predict(X_train_std)

# Evaluasi akurasi training data
acc_train_std = accuracy_score(y_train, y_train_std_pred)

# Prediksi test data yang telah di standarisasi
y_test_std_pred = gnb_std.predict(X_test_std)

# Evaluasi akurasi testing data
acc_test_std = accuracy_score(y_test, y_test_std_pred)

# Print hasil evaluasi
print(f'Hasil akurasi data training terstandarisasi: {acc_train_std}')
print(f'Hasil akurasi data testing terstandarisasi: {acc_test_std}')

Hasil akurasi data training terstandarisasi: 0.7895791583166333
Hasil akurasi data testing terstandarisasi: 0.7534883720930232


## Kesimpulan

Jika diperhatikan, tidak terjadi perubahan yang signifikan antara model dari nilai asli dengan model dengan nilai yang telah di standarisasi, terlebih pada hasil dengan menggunakan data test. Hal ini dikarenakan, Naive Bayes bukan jenis algoritma klasifikasi yang mengandalkan jarak, namun probabilitas. Mean dan standar deviasi mungkin berubah, namun probabilitas akan menghasilkan nilai yang sama